# Advent of code 2023

Solutions are my own, if any external source including hints have been used it shall be mentioned and linked.


## Part 1

----- Day 4: Scratchcards ---

The Elf leads you over to the pile of colorful cards. There, you discover dozens of scratchcards, all with their opaque covering already scratched off. Picking one up, it looks like each card has two lists of numbers separated by a vertical bar (|): a list of winning numbers and then a list of numbers you have. You organize the information into a table (your puzzle input).

As far as the Elf has been able to figure out, you have to figure out which of the numbers you have appear in the list of winning numbers. The first match makes the card worth one point and each match after the first doubles the point value of that card.

For example:
```
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
```
In the above example, card 1 has five winning numbers (41, 48, 83, 86, and 17) and eight numbers you have (83, 86, 6, 31, 17, 9, 48, and 53). Of the numbers you have, four of them (48, 83, 17, and 86) are winning numbers! That means card 1 is worth 8 points (1 for the first match, then doubled three times for each of the three matches after the first).

    Card 2 has two winning numbers (32 and 61), so it is worth 2 points.
    Card 3 has two winning numbers (1 and 21), so it is worth 2 points.
    Card 4 has one winning number (84), so it is worth 1 point.
    Card 5 has no winning numbers, so it is worth no points.
    Card 6 has no winning numbers, so it is worth no points.

So, in this example, the Elf's pile of scratchcards is worth 13 points.

Take a seat in the large pile of colorful cards. How many points are they worth in total?

In [1]:
from dataclasses import dataclass
from typing import Generator
from collections import deque

@dataclass
class Card:
    id_: int
    my_numbers : set[int]
    win_numbers: set[int]
    points:int= 0

    @staticmethod
    def _parse_id(row:str)->int:
        return int(row.split(":")[0].split()[-1])
    
    @staticmethod
    def _parse_win_numbers(row:str)->list[int]:
        temp = row.split("|")[0].split(":")[-1].split()
        return set(int(num) for num in temp)
    
    @staticmethod
    def _parse_my_numbers(row:str)->list[int]:
        temp = row.split("|")[-1].split()
        return set(int(num) for num in temp)
    
    @staticmethod
    def parse_card(row:str)->'Card':
        return Card(
            id_=Card._parse_id(row),
            my_numbers=Card._parse_my_numbers(row),
            win_numbers=Card._parse_win_numbers(row)
        )

    def __post_init__(self):
        """Initializes the card matched numbers, points and copies won"""
        self.get_matched_numbers() 
        self.get_card_points() 
        self.get_card_copies() # added p2

    def get_card_copies(self):
        "keep a list of card num ids of the copies won"
        L = len(self.matched_numbers)
        self.card_copies = [i for i in range(self.id_ + 1, self.id_ + 1 + L)]       
    
    def get_matched_numbers(self):
        self.matched_numbers = self.win_numbers & self.my_numbers 

    def get_card_points(self):
        if self.matched_numbers:
            L = len(self.matched_numbers)
            if L == 1:
                self.points = 1
            else:
                points = 1
                for _ in range(L-1):
                    points *=2
                self.points = points

def parse_puzzle(puzzle:str)->Generator[Card, None, None]:
    for row in puzzle.splitlines():
        yield Card.parse_card(row=row)


def part1(puzzle:str)->int:
    cards = parse_puzzle(puzzle=puzzle)
    return sum(
        card.points
        for card in cards
    )



In [2]:
row = "Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53"
Card.parse_card(row=row).points == 8

TEST = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

assert part1(puzzle=TEST) == 13

## Part 2

In [3]:
def part2(puzzle:str):
    """breadth first search on the scratchcard children (copies)"""
    cards = list(parse_puzzle(puzzle=puzzle))
    queue = deque(cards)  # Initialise queue
    counter = len(queue) # account for original cards
    while queue:
        # bfs start from the root card1
        current_node = queue.popleft()
        # print(current_node.card_copies)
        # Dequeue the first card
        # Enqueue all children (card copies) of the current node
        
        for child_idx in current_node.card_copies:
            queue.append(cards[child_idx-1])  # cards start count in 1 
            counter += 1 # count all cards added
    return counter

assert part2(puzzle=TEST) == 30

## Solutions

In [4]:
with open("puzzle_input/day04.txt") as file:
    puzzle = file.read()

print("part1", part1(puzzle=puzzle))
print("part2", part2(puzzle=puzzle))
# print("part2", sum(part2(puzzle=puzzle, numbers=numbers)))

part1 25010


part2 9924412
